In [7]:
import pandas as pd
import ast
import random

ds = pd.read_csv("./perfect_datasets/perfect_dataset_v2.csv")
ds_overview = pd.read_csv("./perfect_datasets/perfect_dataset_v1_overview.csv")
similarity = pd.read_csv("./perfect_datasets/perfect_datasets_v4.csv")

temp = pd.DataFrame()
temp["title"] = similarity["name"].str.lower().str.split()
temp["title"] = similarity["name"].apply(lambda x: x.replace(":", " :"))

for_sugg = pd.DataFrame()
for_sugg["title"] = similarity["name"].str.lower().str.split()
for_sugg["title"] = similarity["name"].apply(lambda x: x.replace("", ""))

movie_sugg = []
c = 0

def recommend(movie):
    global c
    global movie_sugg
    if(ds[ds["title"].str.lower() == movie.lower()].empty):
        for i in temp["title"]:
            tmp = i.split(",")
            movi = tmp[0].lower().split()
            if(movie.lower() in movi) and c==0:
                movie_sugg.append(tmp[0].replace(" :", ":"))
                print("sug1" + str(movie_sugg))
            if(movie.lower() in movi) and c==1:
                movie_sugg = (tmp[0].replace(" :", ":"))
                print("sug2" + str(movie_sugg))
        print("did you mean: ", movie_sugg, end="\n\n") if c<=1 else print("No movie found")
        c += 1
        return
    elif(movie == ""):
        print("Enter a movie name")
        return
    
    for_sugg["title"] = for_sugg["title"].apply(lambda x: x.split(","))
    
    index = ds[(ds["title"].str.lower()) == movie.lower()].index[0]

    moviess = for_sugg["title"][index][1:]
    rndm = []
    for i in range(0,5):
        rd = random.randint(1,10)
        rndm.append(rd if rd not in rndm else random.randint(1,10))
    print(rndm)
    for i in rndm:
        index2 = ds[(ds["title"].str.lower()) == moviess[i].lower()].index[0]
        print("Movie Id:",ds.iloc[index2]["movie_id"])
        print("Movie Name:",moviess[i].replace(" :", ":"))
        print("Movie Overview:",ds_overview.iloc[index2]["overview"], end="\n\n")

inp = str(input("Enter the movie name: "))
recommend(inp)

if movie_sugg != []:
        movie_re_input = str(input("Enter the movie name: "))
        recommend(movie_re_input)


UnboundLocalError: cannot access local variable 'c' where it is not associated with a value